# Introduction

This notebook is a simple interactive model of emissions at the Mischabel and Windgällen huts.

The intent is to allow us explore different scenarios and to estimate the impact of possible interventions (i.e., is a given change worthwhile?).

The model focuses on food consumption, transportation (helicopter), and, where available, possibly avoidable fuel consumption (e.g., use of the generator at Mischabelhütte). It assumes that most electricity for lighting (and heating/dehumidification in winter) comes from renewables [CC 240126].

For now the model ignores fuel used for cooking at Windgällen because I could not find data. This consumption is unavoidable, but it could be an important part of the denominator when measuring the overall percentage impact of planned emissions reductions. The model does include all liquid fuel and wood use at Mischabel.

# Data sources

The model in this notebook is based on the following data and sources.

## Hut visitation data

| Year | Hut        | Overnights | Meal Sales (k CHF) | Source               |
|------|------------|------------|--------------------|----------------------|
| 2021 | Mischabel  | 2000       | n/a                | GV 220506            |
| 2021 | Windgallen | 2700       | >300               | GV 220506            |
| 2022 | Mischabel  | 3100       | n/a                | GV 230616, CC 221026 |
| 2022 | Windgällen | 3450       | 400 (?)            | CC 221026, CC 230930 |
| 2023 | Mischabel  | 3200       | n/a                | CC 230930, CC 240126 |
| 2023 | Windgällen | 3750       | n/a                | CC 230930, CC 240126 |
| 2024 | Windgällen | 3223       | n/a                | Hut wardens          |

## Food

### Windgällen

The following data is for 2024.

* 2240 hot lunches, 867 cold lunches, 2350 daytime cakes/desserts. ~50% of lunches are vegetarian.
* 3223 dinners (overnight guests), of which 480 (15%) vegetarian.
* Meat served is almost exclusively locally sourced pork, 180 g / person.
* Drinks:
  * ~670 L soft drinks in 1.5 L PET bottles
  * 235 L of Süssmost bag-in-box
  * 312 L of flavored drinks / ice tea from concentrate
  * 1865 L of Bier and wine, mostly in recyclable glass bottles

### Mischabel

Approximately 10-15% of meals are vegetarian. Most meat is chicken or pork, not beef.

## Fuel

### Windgällen

* 24 x 33 kg gas canisters (2.5 heli loads)
* About half for cooking, half for hot water (dishes, washing)
* Might be able to save a flight with larger PV or solar thermal array for warm water

### Mischabel

* 900 CHF for generator diesel in 2023 [CC 230930], but minimally used [Maria A.]
* 330 kg propane (cooking and warm water)
* 1000 kg wood (cooking and heating)

## Heli operations

### Windgällen

* Weekly resupply flight (~20 weeks) + 7 flights at season start + 2 flights at season end
* ~Half of flights from Erstfeld (~9-minute R/T, 520 kg load), ~half from Bristen (even closer, 750 kg load)
* All flights full in both directions except at start and end of season
* Only ~100 kg of fresh vegetables/meat per flight due to limited storage/refrigeration space

### Mischabel

* ~20 heli flights/season (7-10 at startup, then ~1/week for the season) [Maria A.]
* Extra flights at start of season because there is no over-winter storage, so at end of season drinks etc. are flown down.

## Food and drink-related emissions data

Data on food-related emissions across the supply chain is from [Our World in Data](https://ourworldindata.org):
* https://ourworldindata.org/environmental-impacts-of-food
* https://ourworldindata.org/grapher/food-emissions-supply-chain
which is based on [Poore and Nemecek, 2018](https://doi.org/10.1126/science.aaq0216), the largest meta-analysis of food systems to date.

Additional data for bottled drinks comes from [Carbon Cloud](https://carboncloud.com) and [Da Silva, 2022](https://doi.org/10.1016/j.clcb.2022.100021). The impact of different containers (kegs, bottles, etc.) is based on [Bowler et al., 2014](https://doi.org/10.1016/j.jclepro.2023.140181) and [Cimini and Moresi, 2016](https://doi.org/10.1016/j.jclepro.2015.06.063). The authors provide a range of estimates: I have chosen middle-of-the-road values. Differences in packaging types are not incorporated further into heli emissions (e.g., packing density seenms like a minor factor overall).

## Transportation emissions data

Data on helicopter performance comes from [Air Zermatt](https://air-zermatt.ch) and the Eurocopter [technical data sheet](https://www.aviatorsdatabase.com/wp-content/uploads/2013/07/AS350-B3.pdf)

Fuel emissions factors (for jet kerosene) are from the [GHG Protocol](https://ghgprotocol.org/sites/default/files/2024-05/Emission_Factors_for_Cross_Sector_Tools_V2.0_0.xlsx).

The current heli model uses the manufacturer's fuel consumption "at recommended cruise speed". A more sophisticated approach is available in the [Guidance on the Determination of Helicopter Emissions](https://www.bazl.admin.ch/dam/bazl/de/dokumente/Fachleute/Regulationen_und_Grundlagen/guidance_on_the_determinationofhelicopteremissions.pdf.download.pdf/guidance_on_the_determinationofhelicopteremissions.pdf) from the Swiss Bundesamt für Zivilluftfahrt (BFZL).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Model parameters

# Diesel generators
fuel_emissions_kg_co2 = {
    # all data from https://ghgprotocol.org/sites/default/files/2024-05/Emission_Factors_for_Cross_Sector_Tools_V2.0_0.xlsx
    'diesel_per_l': 2.9,
    'kerosene_per_kg': 3.2,
    'propane_per_kg': 2.8,
    'wood_per_kg': 1.7,
}

def heli_emissions(num_flights, flight_duration_h):
    # The Eurocopter data sheet cites a fuel burn rate of 150 kg/h for the AS350 B3.
    # So we used to compute emissions as (burn rate) * (flight duration) * (kg CO2 per kg kerosene).
    # However, the ETH/PSI Mobitool v3.0 (https://www.mobitool.ch/) dataset cites
    # 966 kg CO2 / h for single-engine helicopters.
    # This is about double of our data above (150 * 3.1 = 465 kg CO2 / h).
    # So replacing the calculation above with this more pessimistic value.
    return num_flights * flight_duration_h * 966

def fuel_emissions(amount, fuel_type):
    return amount * fuel_emissions_kg_co2[fuel_type]



In [ ]:
# Food emissions data, all from https://ourworldindata.org/environmental-impacts-of-food

"""
food_kg_co2 = {
    # [ kg CO2e per kg, per 100g protein, per 1000 kcal ], for commodities
    'beef': [99, 50, 36],
    'cheese': [24, 11, 6.2],
    'chicken': [10, 5.7, 5.3],
    'grains': [1.6, 1.3, 0.6],
    'lamb': [40, 20, 12],
    'legumes': [1.8, 0.8, 0.5],
    'pork': [12, 7.6, 5.2],
    'tofu': [3.2, 2.0, 1.2],
    'vegetables': [0.9, 4.0, 3.0],
}
"""

food_types = {
    # [ kg CO2e per kg, per 100g protein, per 1000 kcal ], for specific food items.
    'beef': {
        'Beef burger': [53.98, 28.81, 21.91],
        'Beef steak': [129.75, 64.19, 68.10],
    },
    'cheese': {
        'Cheddar cheese': [20.75, 8.72, 5.41],
        'Goat cheese': [19.31, 12.55, 6.22],
        'Parmesan cheese': [24.02, 8.44, 6.37],
    },
    'chicken': {
        'Chicken breast': [9.27, 4.52, 5.83],
    },
    'eggs': [4.44, 3.14, 2.77],
    'grains': {
        'Bread': [0.88, 1.44, 0.41],
        'Muesli': [2.27, 2.35, 0.71],
        'Pasta shells': [1.03, 0.84, 0.29],
        'Rice': [3.93, 9.12, 2.05],
    },
    'lamb': {
        'Lamb (leg)': [30.74, 16.63, 15.18],
        'Lamb chops': [30.90, 17.19, 11.83],
    },
    'legumes': {
        'Beans': [1.37, 0.89, 0.80],
        'Chickpeas': [1.34, 0.96, 0.59],
        'Lentils': [2.54, 2.04, 1.96],
    },
    'meat alternative': {
        'Meat-free burger': [1.02, 1.55, 0.54],
        'Tofu': [1.02, 0.61, 0.54],
    },
    'pork': {
        'Pork chops': [12.16, 7.41, 4.59],
        'Pork sausages': [9.77, 6.91, 3.66],
    },
    'potatoes': [0.5, 2.7, 0.6],
    'vegetables': {
        'Broccoli': [0.90, 3.03, 2.69],
        'Cauliflower': [0.89, 3.56, 2.60],
        'Courgettes': [0.85, 4.50, 3.25],
        'Kale': [0.90, 3.15, 2.64],
        'Spinach': [1.01, 4.08, 3.77],
    },
}

# Compute the mean of the values in the food_types list.
food_kg_co2 = {}
for k in food_types:
    if not isinstance(food_types[k], dict):
        food_kg_co2[k] = food_types[k]
    else:
        food_kg_co2[k] = np.mean(list(food_types[k].values()), axis=0).tolist()

food_kg_co2_per_kg = {k: v[0] for k, v in food_kg_co2.items()}
food_kg_co2_per_100g_protein = {k: v[1] for k, v in food_kg_co2.items()}
food_kg_co2_per_1000kcal = {k: v[2] for k, v in food_kg_co2.items()}

drink_kg_co2_per_kg = {
    'apple juice': 0.5, # PET bottle
    'almond milk': 0.7, # carton
    'beer bottle': 1.3, # https://doi.org/10.1016/j.jclepro.2023.140181
    'beer can': 0.7,    # https://doi.org/10.1016/j.jclepro.2023.140181
    'beer keg': 0.3,    # https://doi.org/10.1016/j.jclepro.2023.140181
    'cow milk': 3.7,
    'soy milk': 0.9,
    'wine': 1.7,
}

def meal_emissions(h, protein_source):
    # Raise error if protein source is not in the list
    if protein_source not in food_kg_co2_per_100g_protein:
        raise ValueError('Protein source not in list')

    portion = h[protein_source.replace(' ', '_') + '_portion']
    e = food_kg_co2_per_100g_protein[protein_source] * portion

    # Let's add 100 g of grains, 80 g of potatoes, and 100 g of vegetables
    e += food_kg_co2_per_kg['grains'] * 0.1 # 1 kg * 0.1 = 100 g
    e += food_kg_co2_per_kg['potatoes'] * 0.08 # 1 kg * 0.08 = 80 g
    e += food_kg_co2_per_kg['vegetables'] * 0.1 # 1 kg * 0.1 = 100 g
    return e

[food_kg_co2, food_kg_co2_per_kg, food_kg_co2_per_100g_protein]

In [ ]:
huts = {
    'mischabel': {
        'visitors_day': 0,
        'visitors_overnight': 3200,
        'heli_resupplies_per_season': 20,
        'heli_flight_duration_h': 10/60,
        'diesel_per_season_l': 500, # Based on 900 CHF per season and 1.8 CHF per liter
        'propane_per_season_kg': 330,
        'wood_per_season_kg': 1000,
        'fraction_vegetarian': 0.1,
        'protein_source': 'chicken',
        'beef_portion': 0.125, # Beef ragout: 1 Kg including sauce for 7-8 people (125 g per person) (max 1/week)
        'lamb_portion': 0.125,
        'pork_portion': 0.125, # Pork: 1 kg for 8-9 people (125 g per person)
        'chicken_portion': 0.2, # Chicken: 1 Kg für 4-5 people (200 g per person)
        'meat_alternative_portion': 0.2,
        'legumes_portion': 0.2,
        'beer_container': 'can',
        'fraction_wine': 0.1,
        'fraction_beer': 0.4,
        'fraction_juice': 0.3,
        'milk_type': 'cow',
    },
    'windgallen': {
        'visitors_day': 5000, # Based on 2024 data (5457 warm + cold + dessert, assuming some people have meal + dessert)
        'visitors_overnight': 3200,
        'heli_resupplies_per_season': 29,
        'heli_flight_duration_h': 9/60,
        'diesel_per_season_l': 0,
        'propane_per_season_kg': 24*33,
        'wood_per_season_kg': 0,
        'fraction_vegetarian': 0.4, # (2500 + 480) / (5000 + 3200)
        'protein_source': 'pork',
        'beef_portion': 0.125,
        'lamb_portion': 0.125,
        'pork_portion': 0.180, # From hut wardens. The others are as for Mischabel.
        'chicken_portion': 0.2,
        'meat_alternative_portion': 0.2,
        'legumes_portion': 0.2,
        'beer_container': 'bottle',
        # Total number of meals is ~8200.
        # * 1865 L of beer and wine. Assume 1/5 of that is wine => 373 L of wine, 1492 L of beer.
        # * At 1/4 L of wine per meal, 373 * 4 / 8200 = 0.2 of meals have wine.
        # * At 1/2 L per meal, 1492 * 2 / 8200 = 0.4 of meals have beer.
        # * 900 L of soft drinks + Most. (Discounting the concentrate.) At 1/2 L per meal, 900 * 2 / 8200 = 0.2 of meals have juice.
        # The remaining 0.2 of meals have water or juice from concentrate.
        'fraction_wine': 0.2,
        'fraction_beer': 0.4,
        'fraction_juice': 0.2,
        'milk_type': 'cow',
    }
}

def total_emissions(hut):
    h = huts[hut]

    e = 0
    e += heli_emissions(h['heli_resupplies_per_season'], h['heli_flight_duration_h'])
    e += fuel_emissions(h['diesel_per_season_l'], 'diesel_per_l')
    e += fuel_emissions(h['propane_per_season_kg'], 'propane_per_kg')
    e += fuel_emissions(h['wood_per_season_kg'], 'wood_per_kg')

    # Dinner and lunches
    num_meals = huts[hut]['visitors_overnight'] + huts[hut]['visitors_day']
    num_vegetarian = int(num_meals * h['fraction_vegetarian'])
    num_meat = num_meals - num_vegetarian
    e += meal_emissions(h, h['protein_source']) * num_meat + meal_emissions(h, 'legumes') * num_vegetarian

    # Let's assume that everyone has just one drink (conservative)
    num_beers = int(num_meals * h['fraction_beer'])
    num_wines = int(num_meals * h['fraction_wine'])
    num_juices = int(num_meals * h['fraction_juice'])
    # Estimating 1/2 liter of beer or 1/2 liter of juice or 1/4 liter of wine per meal
    e += drink_kg_co2_per_kg['beer ' + h['beer_container']] * num_beers * 0.5
    e += drink_kg_co2_per_kg['wine'] * num_wines * 0.25
    e += drink_kg_co2_per_kg['apple juice'] * num_juices * 0.5

    # Let's assume a breakfast with 1 egg (50 g), 100 g bread, 30 g cheese, and 200 ml cow or soy milk
    num_breakfasts = huts[hut]['visitors_overnight']
    e += food_kg_co2_per_kg['eggs'] * 0.05 * num_breakfasts
    e += food_kg_co2_per_kg['grains'] * 0.1 * num_breakfasts
    e += food_kg_co2_per_kg['cheese'] * 0.03 * num_breakfasts
    e += drink_kg_co2_per_kg[h['milk_type'] + ' milk'] * 0.2 * num_breakfasts

    return e

# Compute the total emissions for each hut (rounded to integer)
for hut in huts.keys():
    print(f'Total emissions for {hut}: {total_emissions(hut):.0f} kg CO2')


In [ ]:
from ipywidgets import interactive_output, IntSlider, FloatSlider, Label, RadioButtons, VBox
from IPython.display import display 

e = total_emissions(hut='mischabel')
print(f'Total CO2 Emissions for Mischabel: {e:.0f} kg CO2')
e = total_emissions(hut='windgallen')
print(f'Total CO2 Emissions for Windgällen: {e:.0f} kg CO2')
default_hut = 'mischabel'
h = huts[default_hut]

style = {'description_width': 'initial'}
layout = {'width': '400px'}
radio_hut = RadioButtons(options=['mischabel', 'windgallen'], value=default_hut, description='Hut', style=style, layout=layout)
slider_visitors_day = IntSlider(min=0, max=10000, value=h['visitors_day'], description='Visitors per day', style=style, layout=layout)
slider_visitors_overnight = IntSlider(min=1, max=5000, value=h['visitors_overnight'], description='Visitors overnight', style=style, layout=layout)
slider_heli_resupplies_per_season = IntSlider(min=0, max=40, value=h['heli_resupplies_per_season'], description='Heli resupplies per season', style=style, layout=layout)
slider_diesel_per_season = IntSlider(min=0, max=2000, value=h['diesel_per_season_l'], description='Diesel per season (l)', style=style, layout=layout)
slider_propane_per_season = IntSlider(min=0, max=2000, value=h['propane_per_season_kg'], description='Propane per season (l)', style=style, layout=layout)
slider_wood_per_season = IntSlider(min=0, max=2000, value=h['wood_per_season_kg'], description='Wood per season (kg)', style=style, layout=layout)
slider_fraction_vegetarian = FloatSlider(min=0, max=1, step=0.1, value=h['fraction_vegetarian'], description='Fraction vegetarian', style=style, layout=layout)
radio_protein_source = RadioButtons(options=['beef', 'lamb', 'pork', 'chicken', 'meat alternative'], value=h['protein_source'], description='Protein source', style=style, layout=layout)
radio_beer_container = RadioButtons(options=['bottle', 'can', 'keg'], value=h['beer_container'], description='Beer container', style=style, layout=layout)
slider_fraction_wine = FloatSlider(min=0, max=1, step=0.1, value=h['fraction_wine'], description='Fraction wine', style=style, layout=layout)
slider_fraction_beer = FloatSlider(min=0, max=1, step=0.1, value=h['fraction_beer'], description='Fraction beer', style=style, layout=layout)
slider_fraction_juice = FloatSlider(min=0, max=1, step=0.1, value=h['fraction_juice'], description='Fraction juice', style=style, layout=layout)
radio_milk_type = RadioButtons(options=['cow', 'soy'], value=h['milk_type'], description='Milk type', style=style, layout=layout)
result = Label(value='')

def summary(hut):
    e = total_emissions(hut)
    epp = e / (huts[hut]['visitors_day'] + huts[hut]['visitors_overnight'])
    return f"Total CO2 Emissions for {hut}: {e :.0f} kg CO2 (per visitor: {epp :.2f} kg CO2)"

hc = False
def recalc(
        visitors_day,
        visitors_overnight,
        heli_resupplies_per_season,
        diesel_per_season_l,
        propane_per_season_kg,
        wood_per_season_kg,
        fraction_vegetarian,
        protein_source,
        beer_container,
        fraction_wine,
        fraction_beer,
        fraction_juice,
        milk_type='cow',
            ):
    global hc
    if hc:
        return
    hut = radio_hut.value
    h = huts[hut]
    h['visitors_day'] = visitors_day
    h['visitors_overnight'] = visitors_overnight
    h['heli_resupplies_per_season'] = heli_resupplies_per_season
    h['diesel_per_season_l'] = diesel_per_season_l
    h['propane_per_season_kg'] = propane_per_season_kg
    h['wood_per_season_kg'] = wood_per_season_kg
    h['fraction_vegetarian'] = fraction_vegetarian
    h['protein_source'] = protein_source
    h['beer_container'] = beer_container
    h['fraction_wine'] = fraction_wine
    h['fraction_beer'] = fraction_beer
    h['fraction_juice'] = fraction_juice
    h['milk_type'] = milk_type
    result.value = summary(hut)

def save_hut_values(hut):
    h = huts[hut]
    h['visitors_day'] = slider_visitors_day.value
    h['visitors_overnight'] = slider_visitors_overnight.value
    h['heli_resupplies_per_season'] = slider_heli_resupplies_per_season.value
    h['diesel_per_season_l'] = slider_diesel_per_season.value
    h['propane_per_season_kg'] = slider_propane_per_season.value
    h['wood_per_season_kg'] = slider_wood_per_season.value
    h['fraction_vegetarian'] = slider_fraction_vegetarian.value
    h['protein_source'] = radio_protein_source.value
    h['beer_container'] = radio_beer_container.value
    h['fraction_wine'] = slider_fraction_wine.value
    h['fraction_beer'] = slider_fraction_beer.value
    h['fraction_juice'] = slider_fraction_juice.value
    h['milk_type'] = radio_milk_type.value

def load_hut_values(hut):
    h = huts[hut]
    slider_visitors_day.value = h['visitors_day']
    slider_visitors_overnight.value = h['visitors_overnight']
    slider_heli_resupplies_per_season.value = h['heli_resupplies_per_season']
    slider_diesel_per_season.value = h['diesel_per_season_l']
    slider_propane_per_season.value = h['propane_per_season_kg']
    slider_wood_per_season.value = h['wood_per_season_kg']
    slider_fraction_vegetarian.value = h['fraction_vegetarian']
    radio_protein_source.value = h['protein_source']
    radio_beer_container.value = h['beer_container']
    slider_fraction_wine.value = h['fraction_wine']
    slider_fraction_beer.value = h['fraction_beer']
    slider_fraction_juice.value = h['fraction_juice']
    radio_milk_type.value = h['milk_type']

def on_hut_change(change):
    global hc
    hc = True
    save_hut_values(change['old'])
    load_hut_values(change['new'])
    hc = False
    result.value = summary(change['new'])

out = interactive_output(recalc, {
    'visitors_day': slider_visitors_day,
    'visitors_overnight': slider_visitors_overnight,
    'heli_resupplies_per_season': slider_heli_resupplies_per_season,
    'diesel_per_season_l': slider_diesel_per_season,
    'propane_per_season_kg': slider_propane_per_season,
    'wood_per_season_kg': slider_wood_per_season,
    'fraction_vegetarian': slider_fraction_vegetarian,
    'protein_source': radio_protein_source,
    'beer_container': radio_beer_container,
    'fraction_wine': slider_fraction_wine,
    'fraction_beer': slider_fraction_beer,
    'fraction_juice': slider_fraction_juice,
    'milk_type': radio_milk_type,
})
radio_hut.observe(on_hut_change, names='value')

display(VBox([
            radio_hut,
            slider_visitors_day,
            slider_visitors_overnight,
            slider_heli_resupplies_per_season,
            slider_diesel_per_season,
            slider_propane_per_season,
            slider_wood_per_season,
            slider_fraction_vegetarian,
            radio_protein_source,
            radio_beer_container,
            slider_fraction_wine,
            slider_fraction_beer,
            slider_fraction_juice,
            radio_milk_type,
            result,
            ]), out)
